In [22]:
import pandas as pd
import numpy as np
from scipy.special import expit
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style

style.use("fivethirtyeight")

np.random.seed(123)
n = 100

In [40]:
import sys
sys.path.append('../scripts/')

from data_cleaning import DataCleaning
from data_extraction import DataExtraction
dclean = DataCleaning()
dextract = DataExtraction()


In [42]:
trip_df = dextract.load_data("../data/nb.csv")
trip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536020 entries, 0 to 536019
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Trip ID           536020 non-null  int64 
 1   Trip Origin       536020 non-null  object
 2   Trip Destination  536020 non-null  object
 3   Trip Start Time   534369 non-null  object
 4   Trip End Time     536019 non-null  object
dtypes: int64(1), object(4)
memory usage: 20.4+ MB


In [24]:
miss_info = trip_df.isna().sum()
miss_info

Trip ID                0
Trip Origin            0
Trip Destination       0
Trip Start Time     1651
Trip End Time          1
dtype: int64

There are 1651 Trip which has no record of trip start time but has trip end time. Why the is a record for trip end time but not for trip start time? Do the driver register their end time only to do some tricks or is there system error for those sptecific trip without trip start time? 

In [43]:
order_df = dextract.load_data("../data/driver_locations_during_request.csv")
order_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
 6   created_at     0 non-null        float64
 7   updated_at     0 non-null        float64
dtypes: float64(4), int64(3), object(1)
memory usage: 95.1+ MB


In [26]:
trip_df.head()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


In [28]:
order_df.head()

,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at
0,1,392001,243828,accepted,6.602207,3.270465,NaN,NaN
1,2,392001,243588,rejected,6.592097,3.287445,NaN,NaN
2,3,392001,243830,rejected,6.596133,3.281784,NaN,NaN
3,4,392001,243539,rejected,6.596142,3.280526,NaN,NaN
4,5,392001,171653,rejected,6.609232,3.288800,NaN,NaN


## Merge the tables
Use order_id and Trip ID columns to merge as the two columns are similar

In [29]:
df = order_df.merge(trip_df, how='inner', left_on='order_id',right_on="Trip ID")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1557740 entries, 0 to 1557739
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   id                1557740 non-null  int64  
 1   order_id          1557740 non-null  int64  
 2   driver_id         1557740 non-null  int64  
 3   driver_action     1557740 non-null  object 
 4   lat               1557740 non-null  float64
 5   lng               1557740 non-null  float64
 6   created_at        0 non-null        float64
 7   updated_at        0 non-null        float64
 8   Trip ID           1557740 non-null  int64  
 9   Trip Origin       1557740 non-null  object 
 10  Trip Destination  1557740 non-null  object 
 11  Trip Start Time   1555718 non-null  object 
 12  Trip End Time     1557740 non-null  object 
dtypes: float64(4), int64(4), object(5)
memory usage: 166.4+ MB


In [31]:
df.isna().sum()

id                        0
order_id                  0
driver_id                 0
driver_action             0
lat                       0
lng                       0
created_at          1557740
updated_at          1557740
Trip ID                   0
Trip Origin               0
Trip Destination          0
Trip Start Time        2022
Trip End Time             0
dtype: int64

## Clean DF
Drop 'created_at' and 'updated_at' -- all the entry is empty.

In [35]:

clean_df = dclean.drop_columns(df,['created_at', 'updated_at'])
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1557740 entries, 0 to 1557739
Data columns (total 11 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   id                1557740 non-null  int64  
 1   order_id          1557740 non-null  int64  
 2   driver_id         1557740 non-null  int64  
 3   driver_action     1557740 non-null  object 
 4   lat               1557740 non-null  float64
 5   lng               1557740 non-null  float64
 6   Trip ID           1557740 non-null  int64  
 7   Trip Origin       1557740 non-null  object 
 8   Trip Destination  1557740 non-null  object 
 9   Trip Start Time   1555718 non-null  object 
 10  Trip End Time     1557740 non-null  object 
dtypes: float64(2), int64(4), object(5)
memory usage: 142.6+ MB


In [30]:
plt.figure(figsize=(6,8))
sns.boxplot(y="enem_score", x="Tablet", data=data).set_title('ENEM score by Tablet in Class')
plt.show()

ValueError: Could not interpret input 'Tablet'

<Figure size 600x800 with 0 Axes>